In [1]:
import findspark

In [2]:
findspark.init('/home/saadatullah/spark-3.3.0-bin-hadoop3')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('lr_expamle').getOrCreate()

22/07/31 22:05:45 WARN Utils: Your hostname, saadatullah-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/07/31 22:05:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/31 22:05:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
from pyspark.ml.regression import LinearRegression

In [8]:
data = spark.read.csv('examples/src/main/resources/Customers_Examples.csv' , inferSchema=True , header=True)

In [9]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [21]:
data.show()

+--------------------+--------------------+----------------+------------------+----------------+----------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|     Time on App| Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+----------------+----------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet|  34.4972677251123|12.6556511491668|39.5776680195262|    4.08262063295296|   587.951053968401|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen|  31.9262720263602|11.1094607286826|37.2689588682977|    2.66403418213262|   392.204933444326|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|  33.0009147556427|11.3302780577775|37.1105974421209|    4.10454320237642|   487.547504867472|
|riverarebecca@gma...|1414 David Throug...|     SaddleBrown|  34.30555

In [22]:
data.head(5)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.4972677251123, Time on App=12.6556511491668, Time on Website=39.5776680195262, Length of Membership=4.08262063295296, Yearly Amount Spent=587.951053968401),
 Row(Email='hduke@hotmail.com', Address='4547 Archer CommonDiazchester, CA 06566-8576', Avatar='DarkGreen', Avg Session Length=31.9262720263602, Time on App=11.1094607286826, Time on Website=37.2689588682977, Length of Membership=2.66403418213262, Yearly Amount Spent=392.204933444326),
 Row(Email='pallen@yahoo.com', Address='24645 Valerie Unions Suite 582Cobbborough, DC 99414-7564', Avatar='Bisque', Avg Session Length=33.0009147556427, Time on App=11.3302780577775, Time on Website=37.1105974421209, Length of Membership=4.10454320237642, Yearly Amount Spent=487.547504867472),
 Row(Email='riverarebecca@gmail.com', Address='1414 David ThroughwayPort Jason, OH 22070-1220', Avatar='SaddleBrown', Avg Sessio

In [37]:
for i in data.head(5)[2]:
    print(i)
    print('--------------')

pallen@yahoo.com
--------------
24645 Valerie Unions Suite 582Cobbborough, DC 99414-7564
--------------
Bisque
--------------
33.0009147556427
--------------
11.3302780577775
--------------
37.1105974421209
--------------
4.10454320237642
--------------
487.547504867472
--------------


In [39]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [40]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [44]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent'], outputCol='features')

In [45]:
output = assembler.transform(data)

In [49]:
output.head(2)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.4972677251123, Time on App=12.6556511491668, Time on Website=39.5776680195262, Length of Membership=4.08262063295296, Yearly Amount Spent=587.951053968401, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511])),
 Row(Email='hduke@hotmail.com', Address='4547 Archer CommonDiazchester, CA 06566-8576', Avatar='DarkGreen', Avg Session Length=31.9262720263602, Time on App=11.1094607286826, Time on Website=37.2689588682977, Length of Membership=2.66403418213262, Yearly Amount Spent=392.204933444326, features=DenseVector([31.9263, 11.1095, 37.269, 2.664, 392.2049]))]

In [50]:
final_data_to_work = output.select(['features' , 'Yearly Amount Spent'])

In [51]:
final_data_to_work

DataFrame[features: vector, Yearly Amount Spent: double]

In [52]:
train_data,test_data = final_data_to_work.randomSplit([0.7 , 0.3])

In [54]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                350|
|   mean| 499.64395709607686|
| stddev|  78.00489498476757|
|    min|   275.918420650386|
|    max|   744.221867104715|
+-------+-------------------+



In [55]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                150|
|   mean| 498.54422763779024|
| stddev|  82.55227967864298|
|    min|   256.670582290056|
|    max|   765.518461938837|
+-------+-------------------+



In [56]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [57]:
lr_model = lr.fit(train_data)

22/07/31 22:43:13 WARN Instrumentation: [18af7569] regParam is zero, which might cause numerical instability and overfitting.
22/07/31 22:43:14 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/07/31 22:43:14 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


22/07/31 22:43:14 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [58]:
test_results = lr_model.evaluate(test_data)

In [61]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|1.193711796076968...|
|-1.56319401867222...|
|-2.55795384873636...|
|-2.04636307898908...|
|-4.20641299569979...|
|2.239630703115835...|
|1.864464138634503...|
|4.888534022029489...|
|4.433786671143025...|
|-1.18234311230480...|
|-3.41060513164848...|
|1.767830326571129...|
|-1.64845914696343...|
|3.410605131648481...|
|-9.09494701772928...|
|-1.36424205265939...|
|-8.12860889709554...|
|-4.94537744089029...|
|5.400124791776761...|
|1.620037437533028...|
+--------------------+
only showing top 20 rows



In [62]:
test_results.rootMeanSquaredError

9.847604588712945e-12

In [63]:
test_results.r2

1.0

In [65]:
test_results.r2

1.0

In [66]:
unlabelled_data = test_data.select('features')

In [68]:
unlabelled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.8162006488763...|
|[30.8364326747734...|
|[30.8794843441275...|
|[31.0613251567161...|
|[31.2834474760582...|
|[31.3123495994443...|
|[31.358477192437,...|
|[31.3662121671877...|
|[31.5741380228732...|
|[31.5761319713223...|
|[31.6005122003033...|
|[31.721652360509,...|
|[31.7366356860503...|
|[31.8093003166792...|
|[31.8186165667691...|
|[31.8279790554653...|
|[31.8745516945854...|
|[31.9480174211614...|
|[32.0180740106321...|
+--------------------+
only showing top 20 rows



In [69]:
predicitions = lr_model.transform(unlabelled_data)

In [71]:
predicitions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 408.6403510726151|
|[30.8162006488763...|266.08634094848463|
|[30.8364326747734...|467.50190042699256|
|[30.8794843441275...|490.20659998485706|
|[31.0613251567161...| 487.5554580579062|
|[31.2834474760582...| 591.7810894256446|
|[31.3123495994443...| 463.5914180279224|
|[31.358477192437,...| 495.1759504494701|
|[31.3662121671877...|430.58888255648054|
|[31.5741380228732...| 544.4092721605988|
|[31.5761319713223...| 541.2265839893283|
|[31.6005122003033...| 479.1728514910793|
|[31.721652360509,...|347.77692663187463|
|[31.7366356860503...|496.93344625552857|
|[31.8093003166792...| 536.7718993628501|
|[31.8186165667691...|446.41867337013736|
|[31.8279790554653...| 440.0027475469501|
|[31.8745516945854...|392.28524424627295|
|[31.9480174211614...| 461.9208768928926|
|[32.0180740106321...| 357.7831107452988|
+--------------------+------------